# Flaw Engine: Cloud Trainer (Brute Force Fix) 🚀

### 🏁 Step 1: Clone Repository
Run this to get the latest fixes onto the machine.

In [ ]:
GITHUB_URL = "https://github.com/Gideon-gg101/flaw-engine.git"

import os
import shutil

print("🔥 Wiping old files for a clean start...")
!rm -rf /content/flaw

print(f"Cloning from {GITHUB_URL}...")
!git clone {GITHUB_URL} /content/flaw

if os.path.exists('/content/flaw'):
    print("\n✅ GitHub Sync Successful!")
    %cd /content/flaw
else:
    print("\n❌ FAILED to clone. Check the URL.")

### 🏁 Step 2: Brute Force Build
This cell manually cleans any old cache that might have come from Windows.

In [ ]:
print("Installing base tools...")
!apt-get install -y cmake g++ libomp-dev > /dev/null
!pip install requests torch > /dev/null

import os
import shutil
import glob

if not os.path.exists('/content/flaw'):
    raise Exception("❌ Section 1 must run first.")

# 1. TOTAL WIPE OF LOCAL BUILD CACHE
BUILD_DIR = '/content/flaw/core/build'
if os.path.exists(BUILD_DIR):
    print("Deleting stale build files...")
    shutil.rmtree(BUILD_DIR)
os.makedirs(BUILD_DIR, exist_ok=True)

# 2. Build with Verbose Error Output
%cd {BUILD_DIR}
print("\n--- Running CMake ---")
!cmake ..

print("\n--- Running Make ---")
!make -j$(nproc)

# 3. Deploy
print("\nDeploying library...")
lib_files = glob.glob("flaw_core*.so")
if not lib_files:
    print("❌ Build failed! Look at the 'Running Make' section above for the error.")
else:
    source_lib = os.path.abspath(lib_files[0])
    shutil.copy2(source_lib, '/content/flaw/flaw_core.so')
    shutil.copy2(source_lib, '/content/flaw/ai/flaw_core.so')
    print("✅ Library deployed to root and ai/ folder.")

%cd /content/flaw
print("\n✅ BUILD COMPLETE!")

### 🏁 Step 3: Start Training
Replace the `MASTER_URL` with your local `ngrok` URL.

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"

import sys
import os
if '/content/flaw' not in sys.path: sys.path.insert(0, '/content/flaw')

try:
    import flaw_core
    from ai.selfplay_worker import DistributedWorker
    print("🚀 Everything loaded. Starting...")
    worker = DistributedWorker(MASTER_URL, depth=1)
    worker.run()
except Exception as e:
    print(f"❌ ERROR: {e}")